<a href="https://colab.research.google.com/github/riccardocappi/Text-Adversarial-Attack/blob/adversarial-training/adversarial_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [ ]:
!pip install textattack

## Imports

In [1]:
import textattack
import os
import transformers
from textattack.datasets import HuggingFaceDataset
from textattack import Attacker
from textattack.attack_recipes import BAEGarg2019
from textattack.metrics.attack_metrics import (
    AttackQueries,
    AttackSuccessRate,
    WordsPerturbed,
)
import json
import os

2024-05-10 16:27:03.852549: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-10 16:27:04.886571: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Loading dataset and model
Getting train data in order to find attack transformation to add into the training set

In [2]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

/home/toldo/Desktop/UniPD/NLP/Text-Adversarial-Attack/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Adversarial training BAEGarg2019

In [3]:
attack = BAEGarg2019.build(model_wrapper)
train_dataset = HuggingFaceDataset("imdb", split="train", shuffle=True)
eval_dataset = HuggingFaceDataset("imdb", split="test", shuffle=True)

training_args = textattack.TrainingArgs(
    num_epochs=3,
    num_clean_epochs=0,
    num_train_adv_examples=1,
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    log_to_tb=True,
    parallel=True,
    random_seed=786
)

trainer = textattack.Trainer(
    model_wrapper,
    "classification",
    attack,
    train_dataset,
    eval_dataset,
    training_args
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Loading datasets dataset imdb, split train.
textatt

## Train

In [4]:
trainer.train()

textattack: Writing logs to ./outputs/2024-05-10-16-27-29-519851/train_log.txt.
textattack: Wrote original training args to ./outputs/2024-05-10-16-27-29-519851/training_args.json.
/home/toldo/Desktop/UniPD/NLP/Text-Adversarial-Attack/venv/lib/python3.10/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
textattack: ***** Running training *****
textattack:   Num examples = 25000
textattack:   Num epochs = 3
textattack:   Num clean epochs = 0
textattack:   Instantaneous batch size per device = 8
textattack:   Total train batch size (w. parallel, distributed & accumulation) = 8
textattack:   Gradient accumulation steps = 1
textattack:   Total optimization steps = 9378
textattack: ==========================================================
textattack: Epoch 1
te

Iteration:   0%|          | 0/3126 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 